In [0]:
# Taken from
# https://stackoverflow.com/questions/48750199/google-colaboratory-misleading-information-about-its-gpu-only-5-ram-available
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
  process = psutil.Process(os.getpid())
  print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
  
# Clone repo
!git clone https://github.com/MatchLab-Imperial/keras_triplet_descriptor
  
# Change directory
%cd /content/keras_triplet_descriptor 

# Download data
!wget -O hpatches_data.zip https://imperialcollegelondon.box.com/shared/static/ah40eq7cxpwq4a6l4f62efzdyt8rm3ha.zip
  
# Extract data
!unzip -q ./hpatches_data.zip
!rm ./hpatches_data.zip

import sys
import json
import os
import glob
import keras
from keras import backend as K
from keras import callbacks
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Lambda, Reshape
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization 
from keras.layers import Input, UpSampling2D, concatenate, Subtract  
import time
import tensorflow as tf
import numpy as np
import cv2
import random
from read_data import HPatches, DataGeneratorDesc, hpatches_sequence_folder, DenoiseHPatches, tps
from utils import generate_desc_csv, plot_denoise, plot_triplet
from keras_contrib.losses import DSSIMObjective # to use DSSIM loss 

random.seed(1234)
np.random.seed(1234)
tf.set_random_seed(1234)
%pwd

hpatches_dir = './hpatches'
splits_path = './splits.json'

splits_json = json.load(open(splits_path, 'rb'))
split = splits_json['a']

train_fnames = split['train']
test_fnames = split['test']

seqs = glob.glob(hpatches_dir+'/*')
seqs = [os.path.abspath(p) for p in seqs]   
seqs_train = list(filter(lambda x: x.split('/')[-1] in train_fnames, seqs)) 
seqs_test = list(filter(lambda x: x.split('/')[-1] in test_fnames, seqs)) 

  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Cloning into 'keras_triplet_descriptor'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 181 (delta 0), reused 1 (delta 0), pack-reused 178
Receiving objects: 100% (181/181), 149.87 MiB | 21.25 MiB/s, done.
Resolving deltas: 100% (65/65), done.
Checking out files: 100% (69/69), done.
/content/keras_triplet_descriptor
--2019-03-31 12:47:48--  https://imperialcollegelondon.box.com/shared/static/ah40eq7cxpwq4a6l4f62efzdyt8rm3ha.zip
Resolving imperialcollegelondon.box.com (imperialcollegelondon.box.com)... 107.152.26.197
Connecting to imperialcollegelondon.box.com (imperialcollegelondon.box.com)|107.152.26.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/ah40eq7cxpwq4a6l4f62efzdyt8rm3ha.zip [following]
-

In [0]:
# def get_denoise_model(shape): # Full U-Net [64 128 256 512 1024 512 256 128 64]
#   inputs = Input(shape)
#   conv1 = Conv2D(64, 3, padding = 'same', kernel_initializer = 'he_normal')(inputs)
#   conv1 = BatchNormalization(momentum=0.9, axis=-1)(conv1)
#   activ1 = Activation('relu')(conv1)
#   activ1 = Dropout(0.2)(activ1)
  
#   conv1 = Conv2D(64, 3, padding = 'same', kernel_initializer = 'he_normal')(activ1)
#   conv1 = BatchNormalization(momentum=0.9, axis=-1)(conv1)
#   activ1 = Activation('relu')(conv1)

#   pool1 = MaxPooling2D(pool_size=(2, 2))(activ1)
#   conv2 = Conv2D(128, 3, padding = 'same', kernel_initializer = 'he_normal')(pool1)
#   conv2 = BatchNormalization(momentum=0.9, axis=-1)(conv2)
#   activ2 = Activation('relu')(conv2)
#   activ2 = Dropout(0.2)(activ2)
 
#   conv2 = Conv2D(128, 3, padding = 'same', kernel_initializer = 'he_normal')(activ2)
#   conv2 = BatchNormalization(momentum=0.9, axis=-1)(conv2)
#   activ2 = Activation('relu')(conv2) 
 
#   pool2 = MaxPooling2D(pool_size=(2, 2))(activ2)
#   conv3 = Conv2D(256, 3, padding = 'same', kernel_initializer = 'he_normal')(pool2)
#   conv3 = BatchNormalization(momentum=0.9, axis=-1)(conv3)
#   activ3 = Activation('relu')(conv3) 
#   activ3 = Dropout(0.2)(activ3)

#   conv3 = Conv2D(256, 3, padding = 'same', kernel_initializer = 'he_normal')(activ3)
#   conv3 = BatchNormalization(momentum=0.9, axis=-1)(conv3)
#   activ3 = Activation('relu')(conv3) 

#   pool3 = MaxPooling2D(pool_size=(2, 2))(activ3)
#   conv4 = Conv2D(512, 3, padding = 'same', kernel_initializer = 'he_normal')(pool3)
#   conv4 = BatchNormalization(momentum=0.9, axis=-1)(conv4)
#   activ4 = Activation('relu')(conv4) 
#   activ4 = Dropout(0.2)(activ4)
 
#   conv4 = Conv2D(512, 3, padding = 'same', kernel_initializer = 'he_normal')(activ4)
#   #conv4 = BatchNormalization(momentum=0.9, axis=-1)(conv4)
#   activ4 = Activation('relu')(conv4) 
#   pool4 = MaxPooling2D(pool_size=(2, 2))(activ4)

#   conv5 = Conv2D(1024, 3, padding = 'same', kernel_initializer = 'he_normal')(pool4)
#   #conv5 = BatchNormalization(momentum=0.9, axis=-1)(conv5)
#   activ5 = Activation('relu')(conv5) 
#   conv5 = Conv2D(1024, 3, padding = 'same', kernel_initializer = 'he_normal')(activ5)
#   conv5 = BatchNormalization(momentum=0.9, axis=-1)(conv5)
#   activ5 = Activation('relu')(conv5) 

#   up6 = Conv2D(512, 2, padding = 'same', activation = 'relu', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(activ5)) 
#   merge6 = concatenate([activ4,up6], axis = -1)
#   conv6 = Conv2D(512, 3, padding = 'same', activation = 'relu', kernel_initializer = 'he_normal')(merge6)  
#   conv6 = Conv2D(512, 3, padding = 'same', activation = 'relu', kernel_initializer = 'he_normal')(conv6)

#   up7 = Conv2D(256, 2, padding = 'same', activation = 'relu', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
#   merge7 = concatenate([activ3,up7], axis = -1)
#   conv7 = Conv2D(256, 3, padding = 'same', activation = 'relu', kernel_initializer = 'he_normal')(merge7)
#   conv7 = Conv2D(256, 3, padding = 'same', activation = 'relu', kernel_initializer = 'he_normal')(conv7)

#   up8 = Conv2D(128, 2, padding = 'same', activation = 'relu', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7)) 
#   merge8 = concatenate([activ2,up8], axis = -1)
#   conv8 = Conv2D(128, 3, padding = 'same', activation = 'relu', kernel_initializer = 'he_normal')(merge8)  
#   conv8 = Conv2D(128, 3, padding = 'same', activation = 'relu',  kernel_initializer = 'he_normal')(conv8)

#   up9 = Conv2D(64, 2, padding = 'same', activation = 'relu', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8)) 
#   merge9 = concatenate([activ1,up9], axis = -1)
#   conv9 = Conv2D(64, 3, padding = 'same', activation = 'relu',  kernel_initializer = 'he_normal')(merge9)
#   conv9 = Conv2D(64, 3, padding = 'same', activation = 'relu',  kernel_initializer = 'he_normal')(conv9)
#   conv9 = Conv2D(1, 3, padding = 'same')(conv9)

#   model = Model(input = inputs, output = conv9)  
#  return model

#def get_denoise_model(shape): # U-Net-- [16 32 64 128 64 32 16]
#   inputs = Input(shape)
#   conv1 = Conv2D(32, 3, padding = 'same', kernel_initializer = 'he_normal')(inputs)
#   conv1 = BatchNormalization(momentum=0.6, axis=-1)(conv1)
#   activ1 = Activation('relu')(conv1)
#   activ1 = Dropout(0.3)(activ1)
  
#   conv1 = Conv2D(32, 3, padding = 'same', kernel_initializer = 'he_normal')(activ1)
#   conv1 = BatchNormalization(momentum=0.6, axis=-1)(conv1)
#   activ1 = Activation('relu')(conv1)

#   pool1 = MaxPooling2D(pool_size=(2, 2))(activ1)
#   conv2 = Conv2D(64, 3, padding = 'same', kernel_initializer = 'he_normal')(pool1)
#   conv2 = BatchNormalization(momentum=0.6, axis=-1)(conv2)
#   activ2 = Activation('relu')(conv2)
#   activ2 = Dropout(0.3)(activ2)
 
#   conv2 = Conv2D(64, 3, padding = 'same', kernel_initializer = 'he_normal')(activ2)
#   conv2 = BatchNormalization(momentum=0.6, axis=-1)(conv2)
#   activ2 = Activation('relu')(conv2) 
 
#   pool2 = MaxPooling2D(pool_size=(2, 2))(activ2)
#   conv3 = Conv2D(128, 3, padding = 'same', kernel_initializer = 'he_normal')(pool2)
#   conv3 = BatchNormalization(momentum=0.6, axis=-1)(conv3)
#   activ3 = Activation('relu')(conv3) 
#   activ3 = Dropout(0.3)(activ3)

#   conv3 = Conv2D(128, 3, padding = 'same', kernel_initializer = 'he_normal')(activ3)
#   conv3 = BatchNormalization(momentum=0.6, axis=-1)(conv3)
#   activ3 = Activation('relu')(conv3) 

#   pool3 = MaxPooling2D(pool_size=(2, 2))(activ3)
#   conv4 = Conv2D(256, 3, padding = 'same', kernel_initializer = 'he_normal')(pool3)
#   conv4 = BatchNormalization(momentum=0.6, axis=-1)(conv4)
#   activ4 = Activation('relu')(conv4)  
#   conv4 = Conv2D(256, 3, padding = 'same', kernel_initializer = 'he_normal')(activ4)
#   activ4 = Activation('relu')(conv4) 

#   up5 = Conv2D(128, 2, padding = 'same', activation = 'relu', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(activ4))
#   merge5 = concatenate([activ3,up5], axis = -1)
#   conv5 = Conv2D(128, 3, padding = 'same', activation = 'relu', kernel_initializer = 'he_normal')(merge5)
#   conv5 = Conv2D(128, 3, padding = 'same', activation = 'relu', kernel_initializer = 'he_normal')(conv5)

#   up6 = Conv2D(64, 2, padding = 'same', activation = 'relu', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv5)) 
#   merge6 = concatenate([activ2,up6], axis = -1)
#   conv6 = Conv2D(64, 3, padding = 'same', activation = 'relu', kernel_initializer = 'he_normal')(merge6)  
#   conv6 = Conv2D(64, 3, padding = 'same', activation = 'relu',  kernel_initializer = 'he_normal')(conv6)

#   up7 = Conv2D(32, 2, padding = 'same', activation = 'relu', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6)) 
#   merge7 = concatenate([activ1,up7], axis = -1)
#   conv7 = Conv2D(32, 3, padding = 'same', activation = 'relu',  kernel_initializer = 'he_normal')(merge7)
#   conv7 = Conv2D(32, 3, padding = 'same', activation = 'relu',  kernel_initializer = 'he_normal')(conv7)
#   conv7 = Conv2D(1, 3, padding = 'same')(conv7)

#   model = Model(input = inputs, output = conv7)
#  return model

#def get_denoise_model(shape): U-Net---
#  inputs = Input(shape)
#  conv1 = Conv2D(16, 3, padding = 'same', kernel_initializer = 'he_normal')(inputs)
#  conv1 = BatchNormalization(momentum=0.9, axis=-1)(conv1)
#  activ1 = Activation('relu')(conv1)
#  activ1 = Dropout(0.3)(activ1)
  
#  conv1 = Conv2D(16, 3, padding = 'same', kernel_initializer = 'he_normal')(activ1)
#  conv1 = BatchNormalization(momentum=0.9, axis=-1)(conv1)
#  activ1 = Activation('relu')(conv1)

#  pool1 = MaxPooling2D(pool_size=(2, 2))(activ1)
#  conv2 = Conv2D(32, 3, padding = 'same', kernel_initializer = 'he_normal')(pool1)
#  conv2 = BatchNormalization(momentum=0.9, axis=-1)(conv2)
#  activ2 = Activation('relu')(conv2)
#  activ2 = Dropout(0.3)(activ2)
 
#  conv2 = Conv2D(32, 3, padding = 'same', kernel_initializer = 'he_normal')(activ2)
#  conv2 = BatchNormalization(momentum=0.9, axis=-1)(conv2)
#  activ2 = Activation('relu')(conv2) 

#  pool2 = MaxPooling2D(pool_size=(2, 2))(activ2)
#  conv3 = Conv2D(64, 3, padding = 'same', kernel_initializer = 'he_normal')(pool2)
#  conv3 = BatchNormalization(momentum=0.9, axis=-1)(conv3)
#  activ3 = Activation('relu')(conv3)  
#  conv3 = Conv2D(64, 3, padding = 'same', kernel_initializer = 'he_normal')(activ3)
#  activ3 = Activation('relu')(conv3) 

#  up4 = Conv2D(32, 2, padding = 'same', activation = 'relu', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(activ3)) 
#  merge4 = concatenate([activ2,up4], axis = -1)
#  conv4 = Conv2D(32, 3, padding = 'same', activation = 'relu', kernel_initializer = 'he_normal')(merge4)  
#  conv4 = Conv2D(32, 3, padding = 'same', activation = 'relu',  kernel_initializer = 'he_normal')(conv4)

#  up5 = Conv2D(16, 2, padding = 'same', activation = 'relu', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv4)) 
#  merge5 = concatenate([activ1,up5], axis = -1)
#  conv5 = Conv2D(16, 3, padding = 'same', activation = 'relu',  kernel_initializer = 'he_normal')(merge5)
#  conv5 = Conv2D(16, 3, padding = 'same', activation = 'relu',  kernel_initializer = 'he_normal')(conv5)
#  conv5 = Conv2D(1, 3, padding = 'same')(conv5)

#  model = Model(input = inputs, output = conv5)
#  return model

#def get_denoise_model(shape): U-Net-
  #U-NET with BN in upsampling stage without Dropout. U-Net-
#  inputs = Input(shape)
#  conv1 = Conv2D(16, 3, padding = 'same', kernel_initializer = 'he_normal')(inputs)
#  conv1 = BatchNormalization(momentum=0.9, axis=-1)(conv1)
#  activ1 = Activation('relu')(conv1)
#  activ1 = Dropout(0.5)(activ1)
  
#  conv1 = Conv2D(16, 3, padding = 'same', kernel_initializer = 'he_normal')(activ1)
#  conv1 = BatchNormalization(momentum=0.9, axis=-1)(conv1)
#  activ1 = Activation('relu')(conv1)

#  pool1 = MaxPooling2D(pool_size=(2, 2))(activ1)
#  conv2 = Conv2D(32, 3, padding = 'same', kernel_initializer = 'he_normal')(pool1)
#  conv2 = BatchNormalization(momentum=0.9, axis=-1)(conv2)
#  activ2 = Activation('relu')(conv2)
#  activ2 = Dropout(0.5)(activ2)
 
#  conv2 = Conv2D(32, 3, padding = 'same', kernel_initializer = 'he_normal')(activ2)
#  conv2 = BatchNormalization(momentum=0.9, axis=-1)(conv2)
#  activ2 = Activation('relu')(conv2) 
 
#  pool2 = MaxPooling2D(pool_size=(2, 2))(activ2)
#  conv3 = Conv2D(64, 3, padding = 'same', kernel_initializer = 'he_normal')(pool2)
#  conv3 = BatchNormalization(momentum=0.9, axis=-1)(conv3)
#  activ3 = Activation('relu')(conv3) 
#  activ3 = Dropout(0.5)(activ3)

#  conv3 = Conv2D(64, 3, padding = 'same', kernel_initializer = 'he_normal')(activ3)
#  conv3 = BatchNormalization(momentum=0.9, axis=-1)(conv3)
#  activ3 = Activation('relu')(conv3) 

#  pool3 = MaxPooling2D(pool_size=(2, 2))(activ3)
#  conv4 = Conv2D(128, 3, padding = 'same', kernel_initializer = 'he_normal')(pool3)
#  conv4 = BatchNormalization(momentum=0.9, axis=-1)(conv4)
#  activ4 = Activation('relu')(conv4) 
#  activ4 = Dropout(0.5)(activ4)
 
#  conv4 = Conv2D(128, 3, padding = 'same', kernel_initializer = 'he_normal')(activ4)
#  conv4 = BatchNormalization(momentum=0.9, axis=-1)(conv4)
#  activ4 = Activation('relu')(conv4) 
#  pool4 = MaxPooling2D(pool_size=(2, 2))(activ4)

#  conv5 = Conv2D(256, 3, padding = 'same', kernel_initializer = 'he_normal')(pool4)
#  conv5 = BatchNormalization(momentum=0.9, axis=-1)(conv5)
#  activ5 = Activation('relu')(conv5) 
#  conv5 = Conv2D(256, 3, padding = 'same', kernel_initializer = 'he_normal')(activ5)
#  conv5 = BatchNormalization(momentum=0.9, axis=-1)(conv5)
#  activ5 = Activation('relu')(conv5) 

#  up6 = Conv2D(128, 2, padding = 'same', activation = 'relu', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(activ5)) 
#  merge6 = concatenate([activ4,up6], axis = -1)
#  conv6 = Conv2D(128, 3, padding = 'same', activation = 'relu', kernel_initializer = 'he_normal')(merge6)  
#  conv6 = Conv2D(128, 3, padding = 'same', activation = 'relu', kernel_initializer = 'he_normal')(conv6)

#  up7 = Conv2D(64, 2, padding = 'same', activation = 'relu', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
#  merge7 = concatenate([activ3,up7], axis = -1)
#  conv7 = Conv2D(64, 3, padding = 'same', activation = 'relu', kernel_initializer = 'he_normal')(merge7)
#  conv7 = Conv2D(64, 3, padding = 'same', activation = 'relu', kernel_initializer = 'he_normal')(conv7)

#  #Skipping operations in the penultimate upsampling layer
#  up8 = Conv2D(32, 2, padding = 'same', activation = 'relu', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7)) 
#  merge8 = concatenate([activ2,up8], axis = -1)
#  up9 = Conv2D(32, 2, padding = 'same', activation = 'relu', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(merge8)) 
#  merge9 = concatenate([activ1,up9], axis = -1)
#  conv9 = Conv2D(32, 3, padding = 'same', activation = 'relu',  kernel_initializer = 'he_normal')(merge9)
#  conv9 = Conv2D(32, 3, padding = 'same', activation = 'relu',  kernel_initializer = 'he_normal')(conv9)
#  conv9 = Conv2D(1, 3, padding = 'same')(conv9)
#  model = Model(input = inputs, output = conv9)
#  return model

def get_denoise_model(shape): # DNCNN architecture
  inpt = Input(shape)
  # 1st layer, Conv+relu
  x = Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding='same')(inpt)
  x = Activation('relu')(x)
  # Layers of Conv+BN+relu
  for i in range(5):
      x = Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding='same')(x)
      x = BatchNormalization(axis=-1, epsilon=1e-3)(x)
      x = Activation('relu')(x)   
  # last layer, Conv
  x = Conv2D(filters=1, kernel_size=(3,3), strides=(1,1), padding='same')(x)
  x = Subtract()([inpt, x])   # input - noise
  model = Model(inputs=inpt, outputs=x)
  return model

def plot_history(history, metric = None):
  # Plots the loss history of training and validation (if existing) and a given metric
  if metric != None:
    fig, axes = plt.subplots(2,1)
    axes[0].plot(history.history[metric])
    try:
      axes[0].plot(history.history['val_'+metric])
      axes[0].legend(['Train', 'Val'])
    except:
      pass
    axes[0].set_title('{:s}'.format(metric))
    axes[0].set_ylabel('{:s}'.format(metric))
    axes[0].set_xlabel('Epoch')
    fig.subplots_adjust(hspace=0.5)
    axes[1].plot(history.history['loss'])
    try:
      axes[1].plot(history.history['val_loss'])
      axes[1].legend(['Train', 'Val'])
    except:
      pass
    axes[1].set_title('Model Loss')
    axes[1].set_ylabel('Loss')
    axes[1].set_xlabel('Epoch')
  else: # if no metric specified, only plot the loss function
    plt.plot(history.history['loss'])
    try:
      plt.plot(history.history['val_loss'])
      plt.legend(['Train', 'Val'])
    except:
      pass
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
  
def PSNR_loss(y_true, y_pred):
	max_pixel = 255.0
	return tf.reciprocal(10.0 * tf_log10((max_pixel ** 2) / (K.mean(K.square(y_pred - y_true))))) 

Using TensorFlow backend.


In [0]:
#denoise_generator = DenoiseHPatches(random.sample(seqs_train, 10), batch_size=50)
denoise_generator = DenoiseHPatches(seqs_train, batch_size=50)
#denoise_generator_val = DenoiseHPatches(random.sample(seqs_test, 10), batch_size=50)
denoise_generator_val = DenoiseHPatches(seqs_test, batch_size=50)

shape = (32, 32, 1) # input image dimension
denoise_model = get_denoise_model(shape)
sgd = keras.optimizers.SGD(lr=0.1,momentum=0.9)
#adam = keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-8, decay=0.0, amsgrad=False)
denoise_model.compile(loss='mean_absolute_error', optimizer=sgd, metrics=['mae'])
#denoise_model.compile(loss=DSSIMObjective(), optimizer=adam, metrics=['mae']) use DSSIM loss 
initial_lr = K.get_value(denoise_model.optimizer.lr)
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=0.00001)
epochs = 10
for e in range(epochs):
  denoise_history = denoise_model.fit_generator(generator=denoise_generator, epochs=1, verbose=1, validation_data=denoise_generator_val,callbacks=[reduce_lr])
  denoise_model.save('denoise.h5') 
  !curl -F "file=@denoise.h5" https://file.io

print('Initial Learning Rate: {:.4f}'.format(initial_lr))
print('Final Learning Rate: {:.10f}'.format(K.eval(denoise_model.optimizer.lr)))
plot_history(denoise_history)

100%|██████████| 40/40 [00:41<00:00,  1.22s/it]


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/1
31179/31179 [==============================] - 2094s 67ms/step - loss: 5.1837 - mean_absolute_error: 5.1837 - val_loss: 5.0617 - val_mean_absolute_error: 5.0617
{"success":true,"key":"KIujlO","link":"https://file.io/KIujlO","expiry":"14 days"}Epoch 1/1
24022/31179 [======================>.......] - ETA: 6:43 - loss: 5.0101 - mean_absolute_error: 5.0101Buffered data was truncated after reaching the output size limit.

In [0]:
def get_descriptor_model(shape): # L2Net architecture
  init_weights = keras.initializers.he_normal()
  descriptor_model = Sequential()
  descriptor_model.add(Conv2D(32, 3, padding='same', input_shape=shape, use_bias = True, kernel_initializer=init_weights))
  descriptor_model.add(BatchNormalization(axis = -1))
  descriptor_model.add(Activation('relu'))

  descriptor_model.add(Conv2D(32, 3, padding='same', use_bias = True, kernel_initializer=init_weights))
  descriptor_model.add(BatchNormalization(axis = -1))
  descriptor_model.add(Activation('relu'))

  descriptor_model.add(Conv2D(64, 3, padding='same', strides=2, use_bias = True, kernel_initializer=init_weights))
  descriptor_model.add(BatchNormalization(axis = -1))
  descriptor_model.add(Activation('relu'))

  descriptor_model.add(Conv2D(64, 3, padding='same', use_bias = True, kernel_initializer=init_weights))
  descriptor_model.add(BatchNormalization(axis = -1))
  descriptor_model.add(Activation('relu'))

  descriptor_model.add(Conv2D(128, 3, padding='same', strides=2,  use_bias = True, kernel_initializer=init_weights))
  descriptor_model.add(BatchNormalization(axis = -1))
  descriptor_model.add(Activation('relu'))

  descriptor_model.add(Conv2D(128, 3, padding='same', use_bias = True, kernel_initializer=init_weights))
  descriptor_model.add(BatchNormalization(axis = -1))
  descriptor_model.add(Activation('relu'))
  descriptor_model.add(Dropout(0.3))

  descriptor_model.add(Conv2D(128, 8, padding='valid', use_bias = True, kernel_initializer=init_weights))
  descriptor_model.add(Reshape((128,)))
  return descriptor_model

def margin_loss(x): # Margin ranking loss
  a, p, n = x
  _alpha = 1.0
  positive_distance = K.mean(K.square(a - p), axis=-1)
  negative_distance = K.mean(K.square(a - n), axis=-1)
  return K.expand_dims(K.maximum(0.0, positive_distance - negative_distance + _alpha), axis = 1)

def margin_hnm_loss(x): # Margin ranking loss with Hard Negative Mining
  a, p, n = x
  _alpha = 1.0 # _alpha is the margin parameter
  positive_distance = K.mean(K.square(a - p), axis=-1)
  negative_distance = K.mean(K.square(a - n), axis=-1)
  pn_distance = K.mean(K.square(p - n), axis=-1)
  return K.expand_dims(K.maximum(0.0, positive_distance - K.minimum(negative_distance, pn_distance) + _alpha), axis = 1)

def ratio_loss(x): # Ratio loss
  a, p, n = x
  p_distance = K.mean(K.square(a - p), axis=-1)
  n_distance = K.mean(K.square(a - n), axis=-1)
  pn_distance = K.mean(K.square(p - n), axis=-1)
  hnm_distance = K.minimum(n_distance,pn_distance)
  t_a = K.pow(tf.math.divide(K.exp(p_distance), K.exp(p_distance) + K.exp(hnm_distance)),2)
  t_b = K.pow(1.0 - tf.math.divide(K.exp(hnm_distance), K.exp(p_distance) + K.exp(hnm_distance)),2)
  return K.expand_dims(t_a + t_b, axis = 1)

def ratio_hnm_loss(x): # Ratio loss with Hard Negative Mining
  a, p, n = x
  p_distance = K.mean(K.square(a - p), axis=-1)
  n_distance = K.mean(K.square(a - n), axis=-1)
  t_a = K.pow(tf.math.divide(K.exp(p_distance), K.exp(p_distance) + K.exp(n_distance)),2)
  t_b = K.pow(1.0 - tf.math.divide(K.exp(n_distance), K.exp(p_distance) + K.exp(n_distance)),2)
  return K.expand_dims(t_a + t_b, axis = 1)

from keras.layers import Lambda
shape = (32, 32, 1)
xa = Input(shape=shape, name='a')
xp = Input(shape=shape, name='p')
xn = Input(shape=shape, name='n')
descriptor_model = get_descriptor_model(shape)
ea = descriptor_model(xa)
ep = descriptor_model(xp)
en = descriptor_model(xn)

loss = Lambda(margin_hnm_loss)([ea, ep, en]) # arbitrary loss function as layer (by doing this we define our own loss function)
print(type(ea))
print(type(ep))
print(type(en))
descriptor_model_trip = Model(inputs = [xa, xp, xn], outputs = loss)
#sgd = keras.optimizers.SGD(lr=0.1,momentum=0.9)
#adam = keras.optimizers.Adam(lr=0.00001) #for ratio and ratio hnm
rmsprop = keras.optimizers.RMSprop(lr=0.0001, rho=0.9, epsilon=None, decay=0.0) # for margin and margin hms
descriptor_model_trip.compile(loss='mean_absolute_error', optimizer=rmsprop)
initial_lr = K.get_value(descriptor_model_trip.optimizer.lr)
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=0.000001)

### Descriptor loading and training
# Loading images
hPatches = HPatches(train_fnames=train_fnames, test_fnames=test_fnames, denoise_model=None, use_clean=True)
# Creating training generator
training_generator = DataGeneratorDesc(*hPatches.read_image_file(hpatches_dir, train=1), num_triplets=100000)
# Creating validation generator
val_generator = DataGeneratorDesc(*hPatches.read_image_file(hpatches_dir, train=0), num_triplets=10000)

epochs = 10
for e in range(epochs):
  descriptor_history = descriptor_model_trip.fit_generator(generator=training_generator, epochs=1, verbose=1, validation_data=val_generator, callbacks=[reduce_lr])
  descriptor_model_trip.save('descriptor.h5') 
  !curl -F "file=@descriptor.h5" https://file.io
    
print('Initial Learning Rate: {:.4f}'.format(initial_lr))
print('Final Learning Rate: {:.10f}'.format(K.eval(descriptor_model_trip.optimizer.lr)))
#descriptor_model_trip.save('descriptor.h5') 

<class 'tensorflow.python.framework.ops.Tensor'>
<class 'tensorflow.python.framework.ops.Tensor'>
<class 'tensorflow.python.framework.ops.Tensor'>
Using clean patches
100%|██████████| 116/116 [00:33<00:00,  3.42it/s]


100%|██████████| 100000/100000 [00:01<00:00, 75942.66it/s]


Using clean patches
100%|██████████| 116/116 [00:20<00:00,  4.69it/s]


100%|██████████| 10000/10000 [00:00<00:00, 73341.61it/s]


Epoch 1/1
1999/2000 [============================>.] - ETA: 0s - loss: 0.2883

...

2000/2000 [==============================] - 172s 86ms/step - loss: 0.1051 - val_loss: 0.1678
{"success":true,"key":"ryTdZh","link":"https://file.io/ryTdZh","expiry":"14 days"}Epoch 1/1
1999/2000 [============================>.] - ETA: 0s - loss: 0.1012

100%|██████████| 100000/100000 [00:02<00:00, 45777.07it/s]


2000/2000 [==============================] - 173s 86ms/step - loss: 0.1012 - val_loss: 0.1457


100%|██████████| 10000/10000 [00:00<00:00, 72325.06it/s]


{"success":true,"key":"82mbqa","link":"https://file.io/82mbqa","expiry":"14 days"}Initial Learning Rate: 0.0001
Final Learning Rate: 0.0001000000


In [0]:
# Hardnet
from read_data import HPatches, DataGeneratorDesc, hpatches_sequence_folder, DenoiseHPatches, tps # when using Colab, make sure to replace read_data.py before running this line for HardNet

def get_descriptor_model(shape): # L2Net architecture
  init_weights = keras.initializers.he_normal()
  descriptor_model = Sequential()
  descriptor_model.add(Conv2D(32, 3, padding='same', input_shape=shape, use_bias = True, kernel_initializer=init_weights))
  descriptor_model.add(BatchNormalization(axis = -1))
  descriptor_model.add(Activation('relu'))

  descriptor_model.add(Conv2D(32, 3, padding='same', use_bias = True, kernel_initializer=init_weights))
  descriptor_model.add(BatchNormalization(axis = -1))
  descriptor_model.add(Activation('relu'))

  descriptor_model.add(Conv2D(64, 3, padding='same', strides=2, use_bias = True, kernel_initializer=init_weights))
  descriptor_model.add(BatchNormalization(axis = -1))
  descriptor_model.add(Activation('relu'))

  descriptor_model.add(Conv2D(64, 3, padding='same', use_bias = True, kernel_initializer=init_weights))
  descriptor_model.add(BatchNormalization(axis = -1))
  descriptor_model.add(Activation('relu'))

  descriptor_model.add(Conv2D(128, 3, padding='same', strides=2,  use_bias = True, kernel_initializer=init_weights))
  descriptor_model.add(BatchNormalization(axis = -1))
  descriptor_model.add(Activation('relu'))

  descriptor_model.add(Conv2D(128, 3, padding='same', use_bias = True, kernel_initializer=init_weights))
  descriptor_model.add(BatchNormalization(axis = -1))
  descriptor_model.add(Activation('relu'))
  descriptor_model.add(Dropout(0.3))

  descriptor_model.add(Conv2D(128, 8, padding='valid', use_bias = True, kernel_initializer=init_weights))
  descriptor_model.add(Reshape((128,)))
  return descriptor_model

def _pairwise_distances(Y1, Y2, squared=False): # compute 2D distance metric between all embeddings 
  Y2T = tf.transpose(Y2)
  SY1 = tf.reduce_sum(tf.square(Y1), axis = 1, keep_dims = True)
  SY2 = tf.reduce_sum(tf.square(Y2T), axis = 0, keep_dims = True)
  dot_product = tf.matmul(Y1, Y2T)
  distances = SY1 - 2 * dot_product
  distances = SY2 + distances
  distances = tf.maximum(distances, 1e-16)

  if not squared:
      distances = tf.sqrt(distances)
  return distances  
    
def hardnet_loss(x): # HardNet loss
  anchor,positive=x
  anchor_swap = True
  margin = 1.0
  batch_reduce = 'min'
  loss_type = "triplet_margin"
  eps = 1e-8
  dist_matrix = _pairwise_distances(anchor, positive) + eps
  
  diag = tf.diag(dist_matrix, name = 'dist_diag')
  pos1 = tf.diag_part(dist_matrix)
  diag_off = dist_matrix - diag
  if batch_reduce == 'min':
      min_neg = tf.reduce_min(diag_off, 1)
      if anchor_swap:
          min_neg2 = tf.reduce_min(diag_off, 0)
          min_neg = tf.minimum(min_neg,min_neg2)
      min_neg = min_neg
      pos = pos1
  
  if loss_type == "triplet_margin":
      loss = tf.maximum(margin + pos - min_neg, 0)
  
  loss = tf.reduce_mean(loss, axis=2)
  return loss

from keras.layers import Lambda
shape = (32, 32, 1)
xa = Input(shape=shape, name='a')
xp = Input(shape=shape, name='p')
descriptor_model = get_descriptor_model(shape)
ea = descriptor_model(xa)
ep = descriptor_model(xp)

loss = Lambda(hardnet_loss)([ea, ep]) 
descriptor_model_trip = Model(inputs = [xa, xp], outputs = loss)
sgd = keras.optimizers.SGD(lr=10)
#rmsprop = keras.optimizers.RMSprop(lr=0.00001, rho=0.9, epsilon=None, decay=0.0)
descriptor_model_trip.compile(loss='mean_absolute_error', optimizer=sgd)
initial_lr = K.get_value(descriptor_model_trip.optimizer.lr)
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=0.00001)

### Descriptor loading and training
# Loading images
hPatches = HPatches(train_fnames=train_fnames, test_fnames=test_fnames,denoise_model=None, use_clean=True)# Creating training generator
training_generator = DataGeneratorDesc(*hPatches.read_image_file(hpatches_dir, train = 1), num_triplets = 100000, out_triplets=False, batch_size=128)
# Creating validation generator
val_generator = DataGeneratorDesc(*hPatches.read_image_file(hpatches_dir, train = 0), num_triplets = 10000, out_triplets=False, batch_size=128)

epochs = 10
for e in range(epochs):
  descriptor_history = descriptor_model_trip.fit_generator(generator=training_generator, epochs=1, verbose=1, validation_data=val_generator)
  descriptor_model_trip.save('descriptor.h5') 
  !curl -F "file=@descriptor.h5" https://file.io
    
print('Initial Learning Rate: {:.4f}'.format(initial_lr))
print('Final Learning Rate: {:.10f}'.format(K.eval(model.optimizer.lr)))

In [0]:
generate_desc_csv(descriptor_model, seqs_test, denoise_model, use_clean=False)
%cd /content/keras_triplet_descriptor 

!python ./hpatches-benchmark/hpatches_eval.py --descr-name=custom --descr-dir=/content/keras_triplet_descriptor/out/ --task=verification --delimiter=";"
!python ./hpatches-benchmark/hpatches_results.py --descr=custom --results-dir=./hpatches-benchmark/python/results/ --task=verification

!python ./hpatches-benchmark/hpatches_eval.py --descr-name=custom --descr-dir=/content/keras_triplet_descriptor/out/ --task=matching --delimiter=";"
!python ./hpatches-benchmark/hpatches_results.py --descr=custom --results-dir=./hpatches-benchmark/results/ --task=matching

!python ./hpatches-benchmark/hpatches_eval.py --descr-name=custom --descr-dir=/content/keras_triplet_descriptor/out/ --task=retrieval --delimiter=";"
!python ./hpatches-benchmark/hpatches_results.py --descr=custom --results-dir=./hpatches-benchmark/results/ --task=retrieval

100%|██████████| 40/40 [26:51<00:00, 45.15s/it]


/content/keras_triplet_descriptor

>> Running HPatch evaluation for custom
>> Please wait, loading the descriptor files...
>> Descriptor files loaded.
>> Evaluating verification task
Processing verification task 1/3 : 100% 1000000/1000000 [01:38<00:00, 10141.43it/s]
Processing verification task 2/3 : 100% 1000000/1000000 [01:39<00:00, 10066.90it/s]
Processing verification task 3/3 : 100% 1000000/1000000 [01:39<00:00, 10083.78it/s]
>> Verification task finished in 305 secs  
Verification task results:
Mean Average Precision is 0.772363

>> Running HPatch evaluation for custom
>> Please wait, loading the descriptor files...
>> Descriptor files loaded.
>> Evaluating matching task
 75% 30/40 [01:29<00:41,  4.15s/it]Traceback (most recent call last):
  File "./hpatches-benchmark/hpatches_eval.py", line 58, in <module>
    res = methods[t](descr,splt)
  File "/content/keras_triplet_descriptor/hpatches-benchmark/utils/tasks.py", line 179, in eval_matching
    D = dist_matrix(d_ref,d,descr['di